In [6]:
import cv2
import mediapipe as mp
import pandas as pd

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

data = []
labels = []

current_label = "G"  # Change this when collecting other letters

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract 21 landmark points
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])

            data.append(landmarks)
            labels.append(current_label)

    cv2.imshow("Collecting Data", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to stop
        break

cap.release()
cv2.destroyAllWindows()

# Save collected dataset
df = pd.DataFrame(data)
df['label'] = labels
df.to_csv("hand_sign_data_G.csv", index=False)


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle 

# Load final dataset
file_path = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\alphabet-signs\Alphabetical_hand_sign_data.csv"
df = pd.read_csv(file_path)

# Features (X) and labels (y)
X = df.drop("label", axis=1)
y = df["label"]

# Split into train/test (stratify keeps A–Z balanced)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train model
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Evaluate
print("Accuracy:", model.score(X_test, y_test))
print("Classes learned:", model.classes_)  # should show A–Z

# Save trained model
with open("sign_model.pkl", "wb") as f:
    pickle.dump(model, f)


Accuracy: 0.9839181286549707
Classes learned: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


In [ ]:
import cv2
import mediapipe as mp
import pickle
import numpy as np

# Load trained model
with open("sign_model.pkl", "rb") as f:
    model = pickle.load(f)

# Mediapipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw landmarks
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract landmarks (same as in training)
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])

            # Convert to NumPy array for prediction
            landmarks = np.array(landmarks).reshape(1, -1)

            # Predict class (A–Z)
            prediction = model.predict(landmarks)
            predicted_label = prediction[0]

            # Show result on frame
            cv2.putText(frame, f"Predicted: {predicted_label}", (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Sign Language Recognition", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit
        break

cap.release()
cv2.destroyAllWindows()
